<a href="https://colab.research.google.com/github/yfirdaws/RFM-analysis/blob/main/RFM_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Data sourced from :
https://www.kaggle.com/ilkeryildiz/online-retail-listing


In [72]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [73]:
df=pd.read_csv("/content/online_retail_listing.csv",delimiter=";",encoding="latin-1")

In [74]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,1.12.2009 07:45,"6,95",13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,1.12.2009 07:45,"6,75",13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,1.12.2009 07:45,"6,75",13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,1.12.2009 07:45,"2,1",13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,1.12.2009 07:45,"1,25",13085.0,United Kingdom


In [75]:
df.shape

(1048575, 8)

In [76]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 8 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   Invoice      1048575 non-null  object 
 1   StockCode    1048575 non-null  object 
 2   Description  1044203 non-null  object 
 3   Quantity     1048575 non-null  int64  
 4   InvoiceDate  1048575 non-null  object 
 5   Price        1048575 non-null  object 
 6   Customer ID  811893 non-null   float64
 7   Country      1048575 non-null  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 64.0+ MB


In [77]:
df.describe()

,Quantity,Customer ID
count,1.048575e+06,811893.000000
mean,9.957525e+00,15324.712265
std,1.335187e+02,1697.033034
min,-7.421500e+04,12346.000000
25%,1.000000e+00,13971.000000
50%,3.000000e+00,15260.000000
75%,1.000000e+01,16795.000000
max,7.421500e+04,18287.000000


In [78]:
df.nunique() #checks for unique values

Invoice        52961
StockCode       5304
Description     5697
Quantity        1048
InvoiceDate    47046
Price           2784
Customer ID     5924
Country           43
dtype: int64

In [79]:
df["Customer ID"]=df["Customer ID"].astype("object")


In [80]:
def myfunction(column): #function to replace comma with dot
    if isinstance(column,str):
        a=column.replace(",",".")
        return a

In [81]:
df["Price"] = df["Price"].apply(myfunction) #function applied to price column
df["Price"] = df["Price"].astype(float)

In [82]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,1.12.2009 07:45,6.95,13085,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,1.12.2009 07:45,6.75,13085,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,1.12.2009 07:45,6.75,13085,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,1.12.2009 07:45,2.10,13085,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,1.12.2009 07:45,1.25,13085,United Kingdom


In [83]:
df["Total"] = df["Quantity"]*df["Price"] #Total gives toyal required to calculate  monetary value
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Total
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,1.12.2009 07:45,6.95,13085,United Kingdom,83.4
1,489434,79323P,PINK CHERRY LIGHTS,12,1.12.2009 07:45,6.75,13085,United Kingdom,81.0
2,489434,79323W,WHITE CHERRY LIGHTS,12,1.12.2009 07:45,6.75,13085,United Kingdom,81.0
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,1.12.2009 07:45,2.10,13085,United Kingdom,100.8
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,1.12.2009 07:45,1.25,13085,United Kingdom,30.0


In [84]:
df.groupby("Description").agg({"Quantity":"sum"}).head() #most sold products

,Quantity
Description,
DOORMAT UNION JACK GUNS AND ROSES,179
3 STRIPEY MICE FELTCRAFT,690
4 PURPLE FLOCK DINNER CANDLES,302
50'S CHRISTMAS GIFT BAG LARGE,1875
ANIMAL STICKERS,385


In [85]:
#Drops rows with no customer ID
df.dropna(subset=["Customer ID"],how='all',inplace=True)
df.shape

(811893, 9)

In [86]:
#Replace . with - because datetime cannot be applied on the latter form
def rem(column):
    if isinstance(column,str):
        a=column.replace(".","-")
        return a

df["InvoiceDate"]=df["InvoiceDate"].apply(rem)
df["InvoiceDate"].head()

0    1-12-2009 07:45
1    1-12-2009 07:45
2    1-12-2009 07:45
3    1-12-2009 07:45
4    1-12-2009 07:45
Name: InvoiceDate, dtype: object

In [87]:
df["InvoiceDate"]=pd.to_datetime(df["InvoiceDate"])


In [88]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Total
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-01-12 07:45:00,6.95,13085,United Kingdom,83.4
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-01-12 07:45:00,6.75,13085,United Kingdom,81.0
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-01-12 07:45:00,6.75,13085,United Kingdom,81.0
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-01-12 07:45:00,2.10,13085,United Kingdom,100.8
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-01-12 07:45:00,1.25,13085,United Kingdom,30.0


In [89]:
#What is the most returned product? 
returned = df[df["Invoice"].str.contains("C",na=False)]
returned.sort_values("Quantity", ascending = True).head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Total
587085,C541433,23166,MEDIUM CERAMIC TOP STORAGE JAR,-74215,2011-01-18 10:17:00,1.04,12346,United Kingdom,-77183.60
529729,C536757,84347,ROTATING SILVER ANGELS T-LIGHT HLDR,-9360,2010-02-12 14:23:00,0.03,15838,United Kingdom,-280.80
507225,C536757,84347,ROTATING SILVER ANGELS T-LIGHT HLDR,-9360,2010-02-12 14:23:00,0.03,15838,United Kingdom,-280.80
359669,C524235,21088,SET/6 FRUIT SALAD PAPER CUPS,-7128,2010-09-28 11:02:00,0.08,14277,France,-570.24
359670,C524235,21096,SET/6 FRUIT SALAD PAPER PLATES,-7008,2010-09-28 11:02:00,0.13,14277,France,-911.04


In [90]:
df=df[df["Invoice"].str.contains("C")==False]

In [91]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Total
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-01-12 07:45:00,6.95,13085,United Kingdom,83.4
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-01-12 07:45:00,6.75,13085,United Kingdom,81.0
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-01-12 07:45:00,6.75,13085,United Kingdom,81.0
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-01-12 07:45:00,2.10,13085,United Kingdom,100.8
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-01-12 07:45:00,1.25,13085,United Kingdom,30.0


In [92]:
df.shape

(793380, 9)

In [93]:
df.isnull().sum()


Invoice        0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
Price          0
Customer ID    0
Country        0
Total          0
dtype: int64

## **RECENCY**

Recency:How recently a customer has made a purchase

In [94]:
#What is the most recent purchase made by each customer
df_recency = df.groupby(by='Customer ID',
                        as_index=False)['InvoiceDate'].max()
df_recency

,Customer ID,InvoiceDate
0,12346.0,2011-01-18 10:01:00
1,12347.0,2011-10-31 12:25:00
2,12348.0,2011-09-25 13:13:00
3,12349.0,2011-11-21 09:51:00
4,12350.0,2011-02-02 16:01:00
...,...,...
5858,18283.0,2011-11-30 12:59:00
5859,18284.0,2010-04-10 11:33:00
5860,18285.0,2010-02-17 10:24:00
5861,18286.0,2010-08-20 11:57:00


To get the recency, we create a column CustomerID and lastPurchaseDate. The last purchase date is then saved in the recent_date variable. To then define the recency, a lambda operation is used to ensure that for every row, the last purchased data is taken from the most recent date. This gives us the recency in days of each purchase at the time the csv was generated.

In [95]:

df_recency.columns = ['CustomerID','LastPurchaseDate']
recent_date = df_recency['LastPurchaseDate'].max()
df_recency['Recency'] = df_recency['LastPurchaseDate'].apply(
    lambda x: (recent_date - x).days)
df_recency.head()

,CustomerID,LastPurchaseDate,Recency
0,12346.0,2011-01-18 10:01:00,326
1,12347.0,2011-10-31 12:25:00,40
2,12348.0,2011-09-25 13:13:00,76
3,12349.0,2011-11-21 09:51:00,19
4,12350.0,2011-02-02 16:01:00,311


In [96]:
df_recency.drop(columns=['LastPurchaseDate'],inplace=True)

# **FREQUENCY**

Frequency: How often a customer makes a purchase

We count the number of invoice date each customer has. As we all know, for each purchase made, an invoive is generated. So counting the number of invoice date each ID has can give us an idea on how frequent their purchases were made.

In [97]:
df.duplicated().sum()

25941

In [98]:
frequency_df = df.drop_duplicates().groupby(
    by=['Customer ID'], as_index=False)['InvoiceDate'].count()
frequency_df.columns = ['Customer ID', 'Frequency']
frequency_df.head()

,Customer ID,Frequency
0,12346.0,34
1,12347.0,211
2,12348.0,51
3,12349.0,175
4,12350.0,17


**MONETARY**

Monetary Value: How much money a customer spends on purchases

We look at the total amount of money a customer has spent.

In [99]:
df_monetary=df.groupby(["Customer ID"],as_index=False)["Total"].sum()

In [100]:
df_monetary.columns = ['Customer ID', 'Monetary']
df_monetary.head()

,Customer ID,Monetary
0,12346.0,77556.46
1,12347.0,5408.50
2,12348.0,2019.40
3,12349.0,4428.69
4,12350.0,334.40


In [101]:
rfm = pd.concat([df_recency, frequency_df['Frequency'], df_monetary['Monetary']],  axis=1)
rfm.head()

,CustomerID,Recency,Frequency,Monetary
0,12346.0,326,34,77556.46
1,12347.0,40,211,5408.50
2,12348.0,76,51,2019.40
3,12349.0,19,175,4428.69
4,12350.0,311,17,334.40


SEGMENT BASED ON RFM

In [102]:
#analyse based on best customers..tier 1 in everything,big spenders,low active spenders, churned best customers( (they transacted frequently and spent a lot, but it’s been a long time since they’ve transacted).)

In [103]:
rfm['Monetary'] = rfm['Monetary'].fillna(0)

To segment based on our RFM we get the score for each, These score will further enable us to calculate the RFM score. To calculate the Recency score, Frequency score and monetary score, the use of quartiles were used. These scores bin each column between range 1-4 with 4 being the maximum score. So with this, we can find which customer was best or the least for each column.

In [104]:
rfm["R"] = pd.cut(rfm["Recency"], 
                                       bins=[-1, 
                                         np.percentile(rfm["Recency"], 25), 
                                             np.percentile(rfm["Recency"], 50), 
                                             np.percentile(rfm["Recency"], 75), 
                                            rfm["Recency"].max()], 
                                       labels=[4, 3, 2, 1]).astype("int")

In [105]:
rfm.head()

,CustomerID,Recency,Frequency,Monetary,R
0,12346.0,326,34,77556.46,2
1,12347.0,40,211,5408.50,3
2,12348.0,76,51,2019.40,3
3,12349.0,19,175,4428.69,4
4,12350.0,311,17,334.40,2


In [106]:
rfm["F"] = pd.cut(rfm["Frequency"], 
                                       bins=[-1, 
                                         np.percentile(rfm["Frequency"], 25), 
                                             np.percentile(rfm["Frequency"], 50), 
                                             np.percentile(rfm["Frequency"], 75), 
                                            rfm["Frequency"].max()], 
                                       labels=[1, 2, 3, 4]).astype("int")

In [107]:
rfm.head()

,CustomerID,Recency,Frequency,Monetary,R,F
0,12346.0,326,34,77556.46,2,2
1,12347.0,40,211,5408.50,3,4
2,12348.0,76,51,2019.40,3,2
3,12349.0,19,175,4428.69,4,4
4,12350.0,311,17,334.40,2,1


In [108]:
rfm["M"] = pd.cut(rfm["Monetary"], 
                                       bins=[-1, 
                                         np.percentile(rfm["Monetary"], 25), 
                                             np.percentile(rfm["Monetary"], 50), 
                                             np.percentile(rfm["Monetary"], 75), 
                                            rfm["Monetary"].max()], 
                                       labels=[1,2,3,4]).astype("float")

In [109]:
rfm.head()

,CustomerID,Recency,Frequency,Monetary,R,F,M
0,12346.0,326,34,77556.46,2,2,4.0
1,12347.0,40,211,5408.50,3,4,4.0
2,12348.0,76,51,2019.40,3,2,3.0
3,12349.0,19,175,4428.69,4,4,4.0
4,12350.0,311,17,334.40,2,1,1.0


In [110]:
#labels moving from lowest to highest for f and m 

To calculate the RFM score we can add the Recency,frequency and monetary individual score in the sense that 1+1+1=3 or we can join them together to get 111 where 1 means 1 in recency,1 in frequency, 1 in monetary. I am sure there are different way this can be done. It is all down to how you want your custtomers segmented. You will most likely see the latter being used here rather than summing. I joined mine as individual strings.

In [111]:
#sums rfm score 
'''rfm["R"]  = pd.qcut(rfm["Recency"], 5, labels = [ 4 , 3, 2, 1])
rfm["F"]= pd.qcut(rfm["Frequency"].rank(method="first"),5, labels=[1,2,3,4])
rfm["M"] = pd.qcut(rfm['Monetary'], 5, labels = [4,3,2,1])
rfm.head()'''

'rfm["R"]  = pd.qcut(rfm["Recency"], 5, labels = [ 4 , 3, 2, 1])\nrfm["F"]= pd.qcut(rfm["Frequency"].rank(method="first"),5, labels=[1,2,3,4])\nrfm["M"] = pd.qcut(rfm[\'Monetary\'], 5, labels = [4,3,2,1])\nrfm.head()'

In [112]:
'''rfm["RFM_SCORE"] = (rfm['R'].astype(str) + 
                    rfm['F'].astype(str) + 
                    rfm['M'].astype(str))'''
  

'rfm["RFM_SCORE"] = (rfm[\'R\'].astype(str) + \n                    rfm[\'F\'].astype(str) + \n                    rfm[\'M\'].astype(str))'

In [113]:
# Calculate RFM_Score
rfm['RFM_Score'] = rfm[['R','F','M']].sum(axis=1)

In [114]:
rfm.head()

,CustomerID,Recency,Frequency,Monetary,R,F,M,RFM_Score
0,12346.0,326,34,77556.46,2,2,4.0,8.0
1,12347.0,40,211,5408.50,3,4,4.0,11.0
2,12348.0,76,51,2019.40,3,2,3.0,8.0
3,12349.0,19,175,4428.69,4,4,4.0,12.0
4,12350.0,311,17,334.40,2,1,1.0,4.0


In [115]:
#The RFM scores give us 4raised to 3 = segments. Which is not easy to work with.
#I am going to work with 7 segments based on the Rfm scores

In [116]:
def rfm_level(df):
    if df['RFM_Score'] >= 9:
        return "Can't Loose Them" #used to purcahse often..need to bring them back
    elif ((df['RFM_Score'] >= 8) and (df['RFM_Score'] < 9)):
        return 'Champions' #bought most recently and often with high purchase
    elif ((df['RFM_Score'] >= 7) and (df['RFM_Score'] < 8)):
        return 'Loyal' #visit more bu less purchase
    elif ((df['RFM_Score'] >= 6) and (df['RFM_Score'] < 7)):
        return 'Potential' #urhase more but no does not vsit often
    elif ((df['RFM_Score'] >= 5) and (df['RFM_Score'] < 6)):
        return 'Promising' #chances of becoming loyal or potential needs to visit and purchase more
    elif ((df['RFM_Score'] >= 4) and (df['RFM_Score'] < 5)):
        return 'Needs Attention' #can turn to churned customers..puhase and visit below average
    else:
        return 'Require Activation'  #churned basically..

In [117]:
# Create a new variable RFM_Level
rfm['RFM_Level'] = rfm.apply(rfm_level, axis=1)

In [118]:
rfm.head()

,CustomerID,Recency,Frequency,Monetary,R,F,M,RFM_Score,RFM_Level
0,12346.0,326,34,77556.46,2,2,4.0,8.0,Champions
1,12347.0,40,211,5408.50,3,4,4.0,11.0,Can't Loose Them
2,12348.0,76,51,2019.40,3,2,3.0,8.0,Champions
3,12349.0,19,175,4428.69,4,4,4.0,12.0,Can't Loose Them
4,12350.0,311,17,334.40,2,1,1.0,4.0,Needs Attention


If you notice, I did not do any visualization. That is because I decided to do that on T ableau instead.To do this, I stored my table in a csv file

In [119]:
rfm.to_csv('RFM_analysis_online_retails.csv')